In [ ]:
from prep_data import load_and_prepare_data, create_datasets

## Data Preparation

We first need to prepare out synthetic data for fine tuning our model. We'd want many more examples than 15, but this is just a demo.

In [2]:
# Load and split the data
data_splits = load_and_prepare_data(file_path="../data/synthetic_insurance_data.json")

Loading dataset from: ../data/synthetic_insurance_data.json

Dataset split complete:
Training examples: 12
Testing examples: 3


In [3]:

# Create datasets
tokenizer, train_dataset, test_dataset = create_datasets(data_splits)


Initializing tokenizer: distilroberta-base


In [4]:

# Let's look at an example from our training dataset
print("\nExample from training dataset:")
example_idx = 0
example = train_dataset[example_idx]

print("Input IDs shape:", example['input_ids'].shape)
print("Attention mask shape:", example['attention_mask'].shape)
print("Label:", example['labels'])


Example from training dataset:
Input IDs shape: torch.Size([69])
Attention mask shape: torch.Size([69])
Label: 1


We now have a dataset that we can use to fine tune our model.

## Fine Tuning RoBERTa

We'll use the `AutoModelForSequenceClassification` class to fine tune our RoBERTa model.